# Team information

|Team-number :| 1|
|:----:|:----:|


|Name|    E-Mail        |matriculation-nr.|
|:----:|:----:|:----:|
|Tamara Scherer| schere21@ads.uni-passau.de|104218|
|Felix Müller| muell518@ads.uni-passau.de|104227|


# Import

In [ ]:
#from label_studio_sdk import Client
#from label_studio_sdk import project
#from label_studio_sdk import project
#import pandas as pd
#LABEL_STUDIO_URL = 'http://132.231.59.226:8080' #this address needs to be the same as the address the label-studio is hosted on.
#API_KEY = '1655a8922f821195356a17a3224c0532b091c61d' #please add your personal API_Key here to get your API_Key follow the Pictures below

#ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
#ls.check_connection()
#pro = project.Project.get_from_id(ls,"1")
#tasks = project.Project.get_labeled_tasks(pro)
#tasks[0]

In [2]:
import pickle
with open("Files/tasks3", 'rb') as f:
    tasks = pickle.load(f)

In [3]:
import re
import pandas as pd

In [9]:
df = pd.DataFrame(columns = ['Text', 'Label',])

for i in range(len(tasks)): 
    for j in range(len(tasks[i]['annotations'][0]['result'])): 
        df = df.append({
            'Text' : tasks[i]['annotations'][0]['result'][j]['value']['text'],
            'Label' : tasks[i]['annotations'][0]['result'][j]['value']['labels']
                        }, ignore_index = True)
df

,Text,Label
0,Good afternoon and thanks a lot for taking my ...,[QID_1]
1,Good afternoon and thanks a lot for taking my...,[Question_1_Company_specific]
2,Good afternoon and thanks a lot for taking my ...,[Question_2_specific]
3,Good afternoon and thanks a lot for taking my ...,[Question_3_neutral]
4,"No, I think that as it relates to the one, I t...",[AID_1]
...,...,...
11289,One of the areas obviously which has been a st...,[Question_3_neutral]
11290,"Well, let's start with the first part of your ...",[AID_1]
11291,"Well, let's start with the first part of your ...",[Answer_1_specific]
11292,"Well, let's start with the first part of your ...",[Answer_2_positive]


# Preprocessing

In [7]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\ts23\AppData\Local\Programs\Python\Python37\python.exe -m pip install --upgrade pip' command.


In [8]:
import string
from gensim.parsing.preprocessing import STOPWORDS
import nltk

# import a stemmer for english words
snowStem = nltk.stem.SnowballStemmer('english')

In [ ]:
# remove numbers 
Sample_Text = re.sub(r'\d+', '', Sample_Text)
print (Sample_Text)

In [ ]:
# remove punctuation
Sample_Text = Sample_Text.translate(str.maketrans('','', string.punctuation))
print(Sample_Text)

In [ ]:
# remove leading and ending white spaces
Sample_Text = Sample_Text.strip()
print(Sample_Text)

In [ ]:
# stemming
Sample_Text = snowStem.stem(Sample_Text)
print(Sample_Text)

In [ ]:
# remove stop words
Sample_Text = ' '.join([w for w in Sample_Text.split() if not(w in STOPWORDS)])
print(Sample_Text)

In [ ]:
Sample_Text.lower()

In [ ]:
def do_preprocessing(text_to_clean, remove_stopwords = True, stemming = False):
    # remove numbers 
    text_to_clean = re.sub(r'\d+', '', text_to_clean)
    # remove punctuation
    text_to_clean = text_to_clean.translate(str.maketrans('','', string.punctuation))
    # remove leading and ending white spaces
    text_to_clean = text_to_clean.strip()
    # transform text to lower case
    text_to_clean = text_to_clean.lower()

    if stemming:
      # stemming
      text_to_clean = snowStem.stem(text_to_clean)
    
    if remove_stopwords:
        # remove stop words
        text_to_clean = ' '.join([w for w in text_to_clean.split() if not(w in STOPWORDS)])
    
    return text_to_clean


for text in my_texts:
  print(do_preprocessing(text))

df.loc[:, 'clean_texts'] = [do_preprocessing(text, remove_stopwords = True) for text in df.content]

WHAT WE HAVE TO DO:

1x Bag-of-Words or tf-idf
1x LDA or LSA
1x GloVe, Word2Vec or Doc2Vec

# Bag-of-Words (BOW)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

count_vec = CountVectorizer()

bow = count_vec.fit_transform(my_texts)
pd.DataFrame(data = bow.toarray(), columns = count_vec.get_feature_names())

# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer()

tfidf = tfidf_vec.fit_transform(tasks[0])
pd.DataFrame(data = tfidf.toarray(), columns = tfidf_vec.get_feature_names())

# LSA

In [ ]:
from scipy.sparse.linalg import svds

num_components = 10
q, s, p = svds(tfidf, k = num_components)

# Doc2Vec